In [1]:
import pickle
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("fakenews_rf")

2023/08/31 12:41:33 INFO mlflow.tracking.fluent: Experiment with name 'fakenews_rf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-artifacts-remote-rollan/2', creation_time=1693464093194, experiment_id='2', last_update_time=1693464093194, lifecycle_stage='active', name='fakenews_rf', tags={}>

In [4]:
import spacy

nlp = spacy.load("en_core_web_lg")

Read and download data

In [5]:
def read_data(filename: str) -> pd.DataFrame:
    """Read data into dataframe"""
    df = pd.read_csv(filename, index_col=0)
    df['vector'] = df['vector'].apply(convert)
    return df

def convert(item):
    item = item.strip()  # remove spaces at the end
    item = item[1:-1]    # remove `[ ]`
    item = np.fromstring(item, sep=' ')  # convert string to `numpy.array`
    return item

In [6]:
df_train = read_data("../../data/train_data.csv")
df_val = read_data("../../data/valid_data.csv")

In [7]:
def add_features(df_train, df_val):
    """"""
    X_train = df_train['vector']
    y_train = df_train['label']

    X_val = df_val['vector']
    y_val = df_val['label']

    X_train = np.stack(X_train)
    y_train = np.stack(y_train)

    X_val = np.stack(X_val)
    y_val = np.stack(y_val)
    return X_train, y_train, X_val, y_val

In [8]:
X_train, y_train, X_val, y_val = add_features(df_train, df_val)

In [9]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    model = RandomForestClassifier(**params, n_jobs=-1)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(model, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 0.10151922665097515


/Users/rollansapyshev/miniconda3/envs/zoomcamp/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
